## code for nqEVI, qEVI, qParGo, and random, completed based on botorch  

In [ ]:
import torch

tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

from botorch.models.gp_regression import SingleTaskGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from botorch.utils.sampling import draw_sobol_samples

from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
#from botorch.utils.multi_objective.box_decomposition import NondominatedPartitioning
%run botorch_utils_multi_objective_box_decomposition.ipynb
#from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement, qNoisyExpectedHypervolumeImprovement
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement, qNoisyExpectedHypervolumeImprovement
from botorch.utils.sampling import sample_simplex

from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qExpectedImprovement, qNoisyExpectedImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume

import time
import warnings
import numpy as np

%run MultiObjectives_version2.ipynb

In [4]:
def generate_initial_data(num_points, problem):
    # generate training data
    lb = problem.lowerbounds
    ub = problem.upperbounds
    #print(lb)
    bds = [lb, ub]
    bds = torch.DoubleTensor(bds)
    train_x = draw_sobol_samples(bounds=bds, n=1, q=num_points, seed=torch.randint(1000000, (1,)).item()).squeeze(0)
    train_x_np = train_x.numpy()
    train_obj_y = np.apply_along_axis(problem.MinMax_evaluate, 1, train_x_np)
    train_obj = torch.from_numpy(train_obj_y)
    #print(')))))',train_obj.shape)
    return train_x, train_obj


def initialize_model(train_x, train_obj):
    # define models for objective and constraint
    model = SingleTaskGP(train_x, train_obj, outcome_transform=Standardize(m=train_obj.shape[-1]))
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    return mll, model


In [5]:
def optimize_qehvi_and_get_observation(model, train_obj, sampler, BATCH_SIZE, problem, standard_bounds):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    partitioning = NondominatedPartitioning(ref_point = problem.ref_point, Y=train_obj)
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=problem.ref_point.tolist(),  # use known reference point
        partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=20,
        raw_samples=1024,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200, "nonnegative": True},
        sequential=True,
    )
    # observe new values
    bds = [problem.lowerbounds, problem.upperbounds]
    bds = torch.DoubleTensor(bds)
    new_x =  unnormalize(candidates.detach(), bounds=bds)

    new_x_np = new_x.numpy()
    new_obj_np = np.apply_along_axis(problem.MinMax_evaluate, 1, new_x_np)
    new_obj = torch.from_numpy(new_obj_np)

    # new_obj = problem(new_x)
    return new_x, new_obj


In [22]:
def optimize_qnehvi_and_get_observation(model, train_x, train_obj, sampler, BATCH_SIZE, problem, standard_bounds):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    partitioning = NondominatedPartitioning(ref_point = problem.ref_point, Y=train_obj)
    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=problem.ref_point.tolist(),  # use known reference point 
        X_baseline=normalize(train_x, standard_bounds),
        prune_baseline=True,  # prune baseline points that have estimated zero probability of being Pareto optimal
        #partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=20,
        raw_samples=1024,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
    
    bds = [problem.lowerbounds, problem.upperbounds]
    bds = torch.DoubleTensor(bds)
    new_x =  unnormalize(candidates.detach(), bounds=bds)
    
    # observe new values 
    #new_x =  unnormalize(candidates.detach(), bounds=problem.bounds)
    new_x_np = new_x.numpy()
    new_obj_np = np.apply_along_axis(problem.MinMax_evaluate, 1, new_x_np)
    new_obj = torch.from_numpy(new_obj_np)
    #new_obj_true = problem(new_x)
    #new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj

In [7]:
def optimize_qparego_and_get_observation(model, train_obj, sampler, BATCH_SIZE, problem, standard_bounds):
    """Samples a set of random weights for each candidate in the batch, performs sequential greedy optimization
    of the qParEGO acquisition function, and returns a new candidate and observation."""
    acq_func_list = []
    for _ in range(BATCH_SIZE):
        weights = sample_simplex(problem.functiondim, **tkwargs).squeeze()
        objective = GenericMCObjective(get_chebyshev_scalarization(weights=weights, Y=train_obj))
        acq_func = qExpectedImprovement(  # pyre-ignore: [28]
            model=model,
            objective=objective,
            best_f=objective(train_obj).max(),
            sampler=sampler,
        )
        acq_func_list.append(acq_func)
    # optimize
    candidates, _ = optimize_acqf_list(
        acq_function_list=acq_func_list,
        bounds=standard_bounds,
        num_restarts=20,
        raw_samples=1024,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
    )
    # observe new values
    bds = [problem.lowerbounds, problem.upperbounds]
    bds = torch.DoubleTensor(bds)
    new_x =  unnormalize(candidates.detach(), bounds=bds)

    new_x_np = new_x.numpy()
    new_obj_np = np.apply_along_axis(problem.MinMax_evaluate, 1, new_x_np)
    new_obj = torch.from_numpy(new_obj_np)

    # new_obj = problem(new_x)
    return new_x, new_obj

In [1]:
def botorch_qehi(N_TRIALS, N_BATCH, MC_SAMPLES, problem, initial_points, BATCH_SIZE, paths):

    # BATCH_SIZE = 4
    standard_bounds = torch.zeros(2, problem.dimension, **tkwargs)
    standard_bounds[1] = 1

    warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    verbose = False

    hvs_qehvi_all = []
    hv = Hypervolume(ref_point=problem.ref_point)
    obj_qehvi_all = []

    # average over multiple trials
    for trial in range(1, N_TRIALS + 1):
        torch.manual_seed(trial)

        print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
        hvs_qehvi = []
        #print(initial_points)

        # call helper functions to generate initial training data and initialize model
        train_x_qehvi, train_obj_qehvi = generate_initial_data(initial_points, problem)
        #print(train_obj_qehvi)

        # compute hypervolume
        mll_qehvi, model_qehvi = initialize_model(train_x_qehvi, train_obj_qehvi)

        # compute pareto front
        pareto_mask = is_non_dominated(train_obj_qehvi)
        pareto_y = train_obj_qehvi[pareto_mask]
        print(pareto_y.shape[-1])

        # compute hypervolume
        volume = hv.compute(pareto_y)
        hvs_qehvi.append(volume)

        print(
            f"\ninitial_points: Hypervolume (qEHVI) = "
            f"({hvs_qehvi[-1]:>4.2f}), ", end="")

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):
            t0 = time.time()
            # fit the models
            fit_gpytorch_model(mll_qehvi)

            # define the qEI and qNEI acquisition modules using a QMC sampler
            qehvi_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)

            # optimize acquisition functions and get new observations
            new_x_qehvi, new_obj_qehvi = optimize_qehvi_and_get_observation(
                model_qehvi, train_obj_qehvi, qehvi_sampler, BATCH_SIZE, problem, standard_bounds
            )

            # update training points
            train_x_qehvi = torch.cat([train_x_qehvi, new_x_qehvi])
            train_obj_qehvi = torch.cat([train_obj_qehvi, new_obj_qehvi])

            # update progress
            # compute pareto front
            pareto_mask = is_non_dominated(train_obj_qehvi)
            pareto_y = train_obj_qehvi[pareto_mask]
            # compute hypervolume
            volume = hv.compute(pareto_y)
            hvs_qehvi.append(volume)

            # reinitialize the models so they are ready for fitting on next iteration
            # Note: we find improved performance from not warm starting the model hyperparameters
            # using the hyperparameters from the previous iteration

            t1 = time.time()

            print(
                f"\nBatch {iteration:>2}: Hypervolume (qEHVI) = "
                f"({hvs_qehvi[-1]:>4.2f}), "
                f"time = {t1 - t0:>4.2f}.", end="")

            mll_qehvi, model_qehvi = initialize_model(train_x_qehvi, train_obj_qehvi)

        obj_qehvi_all.append(train_obj_qehvi)
        hvs_qehvi_all.append(hvs_qehvi)

    ##write output
    import os
    output_dir = paths

    if os.path.exists(os.path.join(output_dir, problem.name + '_qehi_front.txt')):
        os.remove(os.path.join(output_dir, problem.name + '_qehi_front.txt'))

    output_file = open(os.path.join(output_dir, problem.name + '_qehi_front.txt'), "a")
    for n_trial_record in range(N_TRIALS):
        output_file.write("num_trial: " + str(n_trial_record) + '\n')
        obj_qehvi = obj_qehvi_all[n_trial_record].numpy()
        for j in range(len(obj_qehvi)):
            obj_j = obj_qehvi[j]
            for xx in obj_j:
                output_file.write(str(xx) + " ")
            output_file.write('\n')
    output_file.close()

    if os.path.exists(os.path.join(output_dir, problem.name + '_qehi_hypervolume.txt')):
        os.remove(os.path.join(output_dir, problem.name + '_qehi_hypervolume.txt'))

    hv_file = open(os.path.join(output_dir, problem.name + '_qehi_hypervolume.txt'), "a")
    for n_trial_record in range(N_TRIALS):
        hv_file.write("num_trial: " + str(n_trial_record) + '\n')
        hvs_qehvi = hvs_qehvi_all[n_trial_record]
        for j in range(0, len(hvs_qehvi)):
            hv_file.write(str(hvs_qehvi[j]) + "\n")
    hv_file.close()

    return hvs_qehvi_all


In [1]:
def botorch_nqehi(N_TRIALS, N_BATCH, MC_SAMPLES, problem, initial_points, BATCH_SIZE, paths):

    # BATCH_SIZE = 4
    standard_bounds = torch.zeros(2, problem.dimension, **tkwargs)
    standard_bounds[1] = 1

    warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    verbose = False

    hvs_nqehvi_all = []
    hv = Hypervolume(ref_point=problem.ref_point)
    obj_nqehvi_all = []

    # average over multiple trials
    for trial in range(1, N_TRIALS + 1):
        torch.manual_seed(trial)

        print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
        hvs_nqehvi = []

        # call helper functions to generate initial training data and initialize model
        train_x_nqehvi, train_obj_nqehvi = generate_initial_data(initial_points, problem)

        # compute hypervolume
        mll_nqehvi, model_nqehvi = initialize_model(train_x_nqehvi, train_obj_nqehvi)

        # compute pareto front
        pareto_mask = is_non_dominated(train_obj_nqehvi)
        pareto_y = train_obj_nqehvi[pareto_mask]

        # compute hypervolume
        volume = hv.compute(pareto_y)
        hvs_nqehvi.append(volume)

        print(
            f"\ninitial_points: Hypervolume (nqEHVI) = "
            f"({hvs_nqehvi[-1]:>4.2f}), ", end="")

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):
            t0 = time.time()
            # fit the models
            fit_gpytorch_model(mll_nqehvi)

            # define the qEI and nqNEI acquisition modules using a QMC sampler
            nqehvi_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)

            # optimize acquisition functions and get new observations
            new_x_nqehvi, new_obj_nqehvi = optimize_qnehvi_and_get_observation(
                model_nqehvi, train_x_nqehvi, train_obj_nqehvi, nqehvi_sampler, BATCH_SIZE, problem, standard_bounds
            )

            # update training points
            train_x_nqehvi = torch.cat([train_x_nqehvi, new_x_nqehvi])
            train_obj_nqehvi = torch.cat([train_obj_nqehvi, new_obj_nqehvi])

            # update progress
            # compute pareto front
            pareto_mask = is_non_dominated(train_obj_nqehvi)
            pareto_y = train_obj_nqehvi[pareto_mask]
            # compute hypervolume
            volume = hv.compute(pareto_y)
            hvs_nqehvi.append(volume)

            # reinitialize the models so they are ready for fitting on next iteration
            # Note: we find improved performance from not warm starting the model hyperparameters
            # using the hyperparameters from the previous iteration

            t1 = time.time()

            print(
                f"\nBatch {iteration:>2}: Hypervolume (nqEHVI) = "
                f"({hvs_nqehvi[-1]:>4.2f}), "
                f"time = {t1 - t0:>4.2f}.", end="")

            mll_nqehvi, model_nqehvi = initialize_model(train_x_nqehvi, train_obj_nqehvi)

        obj_nqehvi_all.append(train_obj_nqehvi)
        hvs_nqehvi_all.append(hvs_nqehvi)

    ##write output
    import os
    output_dir = paths

    if os.path.exists(os.path.join(output_dir, problem.name + '_nqehi_front.txt')):
        os.remove(os.path.join(output_dir, problem.name + '_nqehi_front.txt'))

    output_file = open(os.path.join(output_dir, problem.name + '_nqehi_front.txt'), "a")
    for n_trial_record in range(N_TRIALS):
        output_file.write("num_trial: " + str(n_trial_record) + '\n')
        obj_nqehvi = obj_nqehvi_all[n_trial_record].numpy()
        for j in range(len(obj_nqehvi)):
            obj_j = obj_nqehvi[j]
            for xx in obj_j:
                output_file.write(str(xx) + " ")
            output_file.write('\n')
    output_file.close()

    if os.path.exists(os.path.join(output_dir, problem.name + '_nqehi_hypervolume.txt')):
        os.remove(os.path.join(output_dir, problem.name + '_nqehi_hypervolume.txt'))

    hv_file = open(os.path.join(output_dir, problem.name + '_nqehi_hypervolume.txt'), "a")
    for n_trial_record in range(N_TRIALS):
        hv_file.write("num_trial: " + str(n_trial_record) + '\n')
        hvs_nqehvi = hvs_nqehvi_all[n_trial_record]
        for j in range(0, len(hvs_nqehvi)):
            hv_file.write(str(hvs_nqehvi[j]) + "\n")
    hv_file.close()

    return hvs_nqehvi_all


In [15]:
def botorch_qparego(N_TRIALS, N_BATCH, MC_SAMPLES, problem, initial_points, BATCH_SIZE, paths):

    # BATCH_SIZE = 4
    standard_bounds = torch.zeros(2, problem.dimension, **tkwargs)
    standard_bounds[1] = 1

    warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    hvs_qparego_all = []
    hv = Hypervolume(ref_point=problem.ref_point)
    obj_qparego_all = []

    # average over multiple trials
    for trial in range(1, N_TRIALS + 1):
        torch.manual_seed(trial)

        print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
        hvs_qparego = []

        # call helper functions to generate initial training data and initialize model
        train_x_qparego, train_obj_qparego = generate_initial_data(initial_points, problem)
        mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_qparego)

        # compute pareto front
        pareto_mask = is_non_dominated(train_obj_qparego)
        pareto_y = train_obj_qparego[pareto_mask]

        # compute hypervolume
        volume = hv.compute(pareto_y)
        hvs_qparego.append(volume)
        print(
            f"\ninitial_points: Hypervolume (parego) = "
            f"({hvs_qparego[-1]:>4.2f}), ", end="")

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):
            t0 = time.time()
            # fit the models
            fit_gpytorch_model(mll_qparego)

            # define the qEI and qNEI acquisition modules using a QMC sampler
            qparego_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)

            # optimize acquisition functions and get new observations
            new_x_qparego, new_obj_qparego = optimize_qparego_and_get_observation(
                model_qparego, train_obj_qparego, qparego_sampler, BATCH_SIZE, problem, standard_bounds
            )

            # update training points
            train_x_qparego = torch.cat([train_x_qparego, new_x_qparego])
            train_obj_qparego = torch.cat([train_obj_qparego, new_obj_qparego])

            # compute pareto front
            pareto_mask = is_non_dominated(train_obj_qparego)
            pareto_y = train_obj_qparego[pareto_mask]
            # compute hypervolume
            volume = hv.compute(pareto_y)
            hvs_qparego.append(volume)

            # reinitialize the models so they are ready for fitting on next iteration
            # Note: we find improved performance from not warm starting the model hyperparameters
            # using the hyperparameters from the previous iteration
            mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_qparego)

            t1 = time.time()

            print(
                f"\nBatch {iteration:>2}: Hypervolume (qparEGO) = "
                f"({hvs_qparego[-1]:>4.2f}), "
                f"time = {t1 - t0:>4.2f}.", end="")

        obj_qparego_all.append(train_obj_qparego)
        hvs_qparego_all.append(hvs_qparego)

    ##write output
    import os
    output_dir = paths

    if os.path.exists(os.path.join(output_dir, problem.name + '_qparEgo_front.txt')):
        os.remove(os.path.join(output_dir, problem.name + '_qparEgo_front.txt'))

    output_file = open(os.path.join(output_dir, problem.name + '_qparEgo_front.txt'), "a")
    for n_trial_record in range(N_TRIALS):
        output_file.write("num_trial: " + str(n_trial_record) + '\n')
        obj_qparego = obj_qparego_all[n_trial_record].numpy()
        for j in range(len(obj_qparego)):
            obj_j = obj_qparego[j]
            for xx in obj_j:
                output_file.write(str(xx) + " ")
            output_file.write('\n')
    output_file.close()

    if os.path.exists(os.path.join(output_dir, problem.name + '_qparEgo_hypervolume.txt')):
        os.remove(os.path.join(output_dir, problem.name + '_qparEgo_hypervolume.txt'))

    hv_file = open(os.path.join(output_dir, problem.name + '_qparEgo_hypervolume.txt'), "a")
    for n_trial_record in range(N_TRIALS):
        hv_file.write("num_trial: " + str(n_trial_record) + '\n')
        hvs_qparego = hvs_qparego_all[n_trial_record]
        for j in range(0, len(hvs_qparego)):
            hv_file.write(str(hvs_qparego[j]) + "\n")
    hv_file.close()

    return hvs_qparego_all

In [16]:
def botorch_random(N_TRIALS, N_BATCH, MC_SAMPLES, problem, initial_points, BATCH_SIZE, paths):

    # BATCH_SIZE = 4
    standard_bounds = torch.zeros(2, problem.dimension, **tkwargs)
    standard_bounds[1] = 1

    warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    hvs_random_all = []
    hv = Hypervolume(ref_point=problem.ref_point)
    obj_random_all = []

    # average over multiple trials
    for trial in range(1, N_TRIALS + 1):
        torch.manual_seed(trial)

        print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
        hvs_random = []

        # call helper functions to generate initial training data and initialize model
        train_x_random, train_obj_random = generate_initial_data(initial_points, problem)

        # compute pareto front
        pareto_mask = is_non_dominated(train_obj_random)
        pareto_y = train_obj_random[pareto_mask]

        # compute hypervolume
        volume = hv.compute(pareto_y)
        hvs_random.append(volume)

        print(
            f"\ninitial_points: Hypervolume (random) = "
            f"({hvs_random[-1]:>4.2f}), ", end="")

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):

            t0 = time.time()

            new_x_random, new_obj_random = generate_initial_data(BATCH_SIZE, problem)

            # update training points
            train_x_random = torch.cat([train_x_random, new_x_random])
            train_obj_random = torch.cat([train_obj_random, new_obj_random])

            # update progress
            # compute pareto front
            pareto_mask = is_non_dominated(train_obj_random)
            pareto_y = train_obj_random[pareto_mask]
            # compute hypervolume
            volume = hv.compute(pareto_y)
            hvs_random.append(volume)


            # reinitialize the models so they are ready for fitting on next iteration
            # Note: we find improved performance from not warm starting the model hyperparameters
            # using the hyperparameters from the previous iteration

            t1 = time.time()

            print(
                f"\nBatch {iteration:>2}: Hypervolume (random) = "
                f"({hvs_random[-1]:>4.2f}), "
                f"time = {t1 - t0:>4.2f}.", end="")

        obj_random_all.append(train_obj_random)
        hvs_random_all.append(hvs_random)


    ##write output
    import os
    output_dir = paths

    if os.path.exists(os.path.join(output_dir, problem.name + '_random_front.txt')):
        os.remove(os.path.join(output_dir, problem.name + '_random_front.txt'))

    output_file = open(os.path.join(output_dir, problem.name + '_random_front.txt'), "a")
    for n_trial_record in range(N_TRIALS):
        output_file.write("num_trial: " + str(n_trial_record) + '\n')
        obj_random = obj_random_all[n_trial_record].numpy()
        for j in range(len(obj_random)):
            obj_j = obj_random[j]
            for xx in obj_j:
                output_file.write(str(xx) + " ")
            output_file.write('\n')

    output_file.close()

    if os.path.exists(os.path.join(output_dir, problem.name + '_random_hypervolume.txt')):
        os.remove(os.path.join(output_dir, problem.name + '_random_hypervolume.txt'))

    hv_file = open(os.path.join(output_dir, problem.name + '_random_hypervolume.txt'), "a")
    for n_trial_record in range(N_TRIALS):
        hv_file.write("num_trial: " + str(n_trial_record) + '\n')
        hvs_random = hvs_random_all[n_trial_record]
        for j in range(0, len(hvs_random)):
            hv_file.write(str(hvs_random[j]) + "\n")
    hv_file.close()

    return hvs_random_all